In [1]:
from sympy import *
init_printing()

Define time variable

In [2]:
t = Symbol('t')

Define parameter symbols

In [3]:
# limb lengths
l_1, l_2, l_3, l_4 = symbols('l_1, l_2, l_3, l_4')

# limb masses
m_1, m_2, m_3 = symbols('m_1, m_2, m_3')

# joint stiffnesses
k_B, k_C = symbols('k_B, k_C')

# joint damping
b_B, b_C = symbols('b_B, b_C')

# joint neutral angles
psi_B, psi_C = symbols('psi_B, psi_C')

# foot angle
phi_D = symbols('phi_E')

# Walking speed
v = symbols('v')

Define coordinate symbols

In [4]:
# limb orientations
theta_1 = Function('theta_1')(t)
theta_2 = Function('theta_2')(t)
theta_3 = Function('theta_3')(t)

# hip height
y = Function('y')(t)

Calculate positions

In [5]:
# rates of change
omega_1 = diff(theta_1)
omega_2 = diff(theta_2)
omega_3 = diff(theta_3)
v = diff(y)

# heel angle
theta_4 = theta_3 - phi_D
omega_4 = diff(theta_4, t)

# joint positions
x_A = 0
y_A = y
x_B = x_A + l_1 * sin(theta_1)
y_B = y_A - l_1 * cos(theta_1)
x_C = x_B + l_2 * sin(theta_2)
y_C = y_B - l_2 * cos(theta_2)
x_D = x_C + l_3 * sin(theta_3)
y_D = y_C - l_3 * cos(theta_3)
x_E = x_C + l_4 * sin(theta_4)
y_E = y_C - l_4 * cos(theta_4)

# joint velocities
u_A = diff(x_A, t)
v_A = diff(y_A, t)
u_B = diff(x_B, t)
v_B = diff(y_B, t)
u_C = diff(x_C, t)
v_C = diff(y_C, t)
u_D = diff(x_D, t)
v_D = diff(y_D, t)
u_E = diff(x_E, t)
v_E = diff(y_E, t)

# Centers of masses
x_1 = x_A + l_1 * sin(theta_1) / 2
y_1 = y_A - l_1 * cos(theta_1) / 2
x_2 = x_B + l_2 * sin(theta_2) / 2
y_2 = y_B - l_2 * cos(theta_2) / 2
x_3 = x_C + l_3 * sin(theta_3) / 2
y_3 = y_C - l_3 * cos(theta_3) / 2

# COM velocities
u_1 = diff(x_1, t)
v_1 = diff(y_1, t)
u_2 = diff(x_2, t)
v_2 = diff(y_2, t)
u_3 = diff(x_3, t)
v_3 = diff(y_3, t)

In [6]:
# Moments of inertia
I_1 = m_1 * l_1**2 / 12
I_2 = m_2 * l_2**2 / 12
I_3 = m_3 * l_3**2 / 12

In [7]:
# read the expression for x_E
x_E

Calculate Lagrangian

In [8]:
# Kinetic Energy
T = m_1 * (u_1**2 + v_1**2) / 2 + m_2 * (u_2**2 + v_2**2) / 2 + m_3 * (u_3**2 + v_3**2) / 2 + I_1 * omega_1**2 / 2 + I_2 * omega_2**2 / 2 + I_3 * omega_3**2 / 2

# Potential Energy
V = k_B * (theta_1 - theta_2 - psi_B)**2 / 2 + k_C * (theta_2 - theta_3 - psi_C)**2 / 2

# Content
D = b_B * (omega_1 - omega_2)**2 / 2 + b_C * (omega_2 - omega_3)**2 / 2

In [9]:
## Constraint equations ###

''' Notes: 
    - This is a section to lay out the constraints/define the "regiemes". I am wondering if for our analysis we should pull set this in a 
       for loop? By that I mean should it be have a matrix where time is the top row and we have a row for each theta. At every time interval 
       we have a measured theta. The below constraints then are only active over their "regieme" and apply to the analysis in their specified
       time domain?
    '''

# Swing

'''Notes:
      - This is the stage where the one leg is swinging while the other leg is standing up going through the cycle.
  
  '''

const_FP_y_A = Eq(y_A, l_4 + l_2*cos(theta_2) + l_1*cos(theta_1))


# Heel_Strike
'''Notes:
            - This is where heel strike contacts the gorund and jhas the following contraints
                - y_E = 0 
                - u_3 = 0
                - v_3 = 0 
                - y_D = ED*sin(theta3) This is inherent to the system will not be set

                Side: Note, should this be a for loop with a matrix? like for t = .1->.25 then have the HS constraints?
                   '''

const_HS_y_E = y_E
const_HS_u_E = u_E + v


# Foot_Planted

'''Notes:
            - This is where the foot is "pulling" across the ground to push the body forward. As we have restricted Y to the 0,0
              position, the foot will be moving around it.
                - y_E = 0 due to the foot being on the ground
                - y_D = 0 due to the foot being on the gorund
                - v_3 = due to the foot not traveling upward 
                   '''

const_FP_y_E = y_E
const_FP_y_D = y_D
const_FP_u_E = u_E + v
const_FP_u_D = u_D + v

# Toe_Lift_Off
'''Notes:
            - This is where the toe is about to kick off of the ground. 
                - y_D = 0 due to the foot being on the gorund

'''

const_TLO_y_D = y_D
const_TLO_u_D = u_D + v
